In [1]:
%matplotlib inline

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

from konlpy.tag import Komoran; tokenizer = Komoran()

import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
corpus_name = "normal chatbot data"
corpus = os.path.join("data", corpus_name)
textfilename = "shuffled_data.txt"

def printLines(file, n=10):
    with open(file, 'r', encoding = 'utf-8') as datafile:
        lines = datafile.readlines()
    for line in lines[:n]:
        print(line)

printLines(os.path.join(corpus, textfilename))
datafile = os.path.join(corpus, textfilename)

아내가 이제 건강해져서 여행도 다니고 있어. 그런 모습을 보는 것이 참 기뻐.	아내가 건강해져서 여행도 다니실 수 있게 되다니 참 다행이에요.

그냥 공무원 친구들의 노력을 생각하면서 인정해주려고. 걔들이 노력한 만큼의 대가를 받는 거겠지 뭐.	당신의 생각대로 잘 돼서 질투심을 덜길 바라요.

생각도 못했는데 현명한 아내를 둬서 너무 고마워.	아내에게 고마운 마음을 어떻게 전하면 좋을까요?

오래 사귄 여자 친구와는 처음 만날 때처럼 떨림은 없지만 편안한 안정감을 느낄 수 있어.	오래 사귄 여자친구에게 편안한 안정감을 느껴서 행복하시군요!

응. 너무 편하다. 다음 달 대출 상황도 연체되지 않게 열심히 일해야겠어.	열심히 일해서 다음 달 대출 상환도 기한을 지키길 바라요.

다 내 잘못이지	아니에요. 너무 자책하지 마세요.

착해서 잘해주는 건지 좋아하는 건지	헷갈린다고 말해보세요.

선생님께 따돌림 사실을 진작 알렸다면 이렇게까지 심각해지지 않았을지도 모를 것 같아. 요새 너무 슬퍼.	일찍 선생님께 말씀 드리지 못한 것이 후회되시나 봐요.

나이를 먹으면서 몸 관리를 소홀하게 했더니 이렇게 빨리 늙어버렸네.	다른 일로 바쁘셨나 봐요. 지금이라도 운동하며 관리 하면 어떨까요?

주말은 근무시간이 아닌데 자꾸 회사 사람들이 불러내서 짜증나.	주말에 회사 사람들 때문에 쉬질 못하여서 화가 나셨군요. 조금 더 자세히 말씀해주시겠어요?



In [4]:
# 기본 단어 토큰 값
PAD_token = 0  # 짧은 문장을 채울(패딩, PADding) 때 사용할 제로 토큰
SOS_token = 1  # 문장의 시작(SOS, Start Of Sentence)을 나타내는 토큰
EOS_token = 2  # 문장의 끝(EOS, End Of Sentence)을 나태는 토큰

class Voc:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3  # SOS, EOS, PAD를 센 것

    def addSentence(self, sentence):
        for word in tokenizer.morphs(sentence):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1

    # 등장 횟수가 기준 이하인 단어를 정리합니다
    def trim(self, min_count):
        if self.trimmed:
            return
        self.trimmed = True

        keep_words = []

        for k, v in self.word2count.items():
            if v >= min_count:
                keep_words.append(k)

        print('keep_words {} / {} = {:.4f}'.format(
            len(keep_words), len(self.word2index), len(keep_words) / len(self.word2index)
        ))

        # 사전을 다시 초기화힙니다
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3 # 기본 토큰을 센 것

        for word in keep_words:
            self.addWord(word)

In [5]:
MAX_LENGTH = 20  # 고려할 문장의 최대 길이

# 유니코드 문자열을 아스키로 변환합니다
# https://stackoverflow.com/a/518232/2809427 참고
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# 소문자로 만들고, 공백을 넣고, 알파벳 외의 글자를 제거합니다
def normalizeString(s):
    hangul = re.compile('[^ ㄱ-ㅣ가-힣 ^☆; ^a-zA-Z.!?;0-9]+')
    result = hangul.sub('', s)
    return result

# 질의/응답 쌍을 읽어서 voc 객체를 반환합니다
def readVocs(datafile, corpus_name):
    print("Reading lines...")
    # 파일을 읽고, 쪼개어 lines에 저장합니다
    lines = open(datafile, encoding='utf-8').\
        read().strip().split('\n')
    # 각 줄을 쪼개어 pairs에 저장하고 정규화합니다
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
    voc = Voc(corpus_name)
    return voc, pairs

# 문장의 쌍 'p'에 포함된 두 문장이 모두 MAX_LENGTH라는 기준보다 짧은지를 반환합니다
def filterPair(p):
    # EOS 토큰을 위해 입력 시퀀스의 마지막 단어를 보존해야 합니다
    return len(tokenizer.morphs(p[0])) < MAX_LENGTH and len(tokenizer.morphs(p[1])) < MAX_LENGTH

# 조건식 filterPair에 따라 pairs를 필터링합니다
def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

# 앞에서 정의한 함수를 이용하여 만든 voc 객체와 리스트 pairs를 반환합니다
def loadPrepareData(corpus, corpus_name, datafile, save_dir):
    print("Start preparing training data ...")
    voc, pairs = readVocs(datafile, corpus_name)
    print("Read {!s} sentence pairs".format(len(pairs)))
    pairs = filterPairs(pairs)
    print("Trimmed to {!s} sentence pairs".format(len(pairs)))
    print("Counting words...")
    for pair in pairs:
        voc.addSentence(pair[0])
        voc.addSentence(pair[1])
    print("Counted words:", voc.num_words)
    return voc, pairs


# voc와 pairs를 읽고 재구성합니다
save_dir = os.path.join("data", "save")
voc, pairs = loadPrepareData(corpus, corpus_name, datafile, save_dir)
# 검증을 위해 pairs의 일부 내용을 출력해 봅니다
print("\npairs:")
for pair in pairs[:10]:

    print(pair)

Start preparing training data ...
Reading lines...
Read 37846 sentence pairs
Trimmed to 21679 sentence pairs
Counting words...
Counted words: 7965

pairs:
['생각도 못했는데 현명한 아내를 둬서 너무 고마워.', '아내에게 고마운 마음을 어떻게 전하면 좋을까요?']
['다 내 잘못이지', '아니에요. 너무 자책하지 마세요.']
['착해서 잘해주는 건지 좋아하는 건지', '헷갈린다고 말해보세요.']
['건강 보조제를 몇 개 챙겨갈까 해.', '친구 분이 얼른 기운내서 쾌차하시길 바랄게요.']
['참 힘들다', '말해줘서 고마워요.']
['장인어른이 많이 편찮으셔서 병원비를 마련하느라 고생 좀 했어.', '장인어른이 편찮으셔서 고생하셨군요. 많이 힘들었겠어요.']
['헤어지고 나니까 휘청거리네여', '정신 차리세요.']
['남자친구와 진도가 보통 어때?', '사람마다 다르겠죠.']
['부모님이 기뻐하실 생각을 하니까 너무 신난다.', '부모님께 알려드릴 생각하니 기쁘시군요.']
['4개월째.', '이별 중인가봐요.']


In [6]:
MIN_COUNT = 2    # 제외할 단어의 기준이 되는 등장 횟수

def trimRareWords(voc, pairs, MIN_COUNT):
    # MIN_COUNT 미만으로 사용된 단어는 voc에서 제외합니다
    voc.trim(MIN_COUNT)
    # 제외할 단어가 포함된 경우를 pairs에서도 제외합니다
    keep_pairs = []
    for pair in pairs:
        input_sentence = pair[0]
        output_sentence = pair[1]
        keep_input = True
        keep_output = True
        # 입력 문장을 검사합니다
        for word in tokenizer.morphs(input_sentence):
            if word not in voc.word2index:
                keep_input = False
                break
        # 출력 문장을 검사합니다
        for word in tokenizer.morphs(output_sentence):
            if word not in voc.word2index:
                keep_output = False
                break

        # 입출력 문장에 제외하기로 한 단어를 포함하지 않는 경우만을 남겨둡니다
        if keep_input and keep_output:
            keep_pairs.append(pair)

    print("Trimmed from {} pairs to {}, {:.4f} of total".format(len(pairs), len(keep_pairs), len(keep_pairs) / len(pairs)))
    return keep_pairs


# voc와 pairs를 정돈합니다
pairs = trimRareWords(voc, pairs, MIN_COUNT)

keep_words 5664 / 7962 = 0.7114
Trimmed from 21679 pairs to 19640, 0.9059 of total


In [7]:
def indexesFromSentence(voc, sentence):
    return [voc.word2index[word] for word in tokenizer.morphs(sentence)] + [EOS_token]


def zeroPadding(l, fillvalue=PAD_token):
    return list(itertools.zip_longest(*l, fillvalue=fillvalue))

def binaryMatrix(l, value=PAD_token):
    m = []
    for i, seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == PAD_token:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

# 입력 시퀀스 텐서에 패딩한 결과와 lengths를 반환합니다
def inputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    padVar = torch.LongTensor(padList)
    return padVar, lengths

# 패딩한 목표 시퀀스 텐서, 패딩 마스크, 그리고 최대 목표 길이를 반환합니다
def outputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    max_target_len = max([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    mask = binaryMatrix(padList)
    mask = torch.ByteTensor(mask)
    padVar = torch.LongTensor(padList)
    return padVar, mask, max_target_len

# 입력 배치를 이루는 쌍에 대한 모든 아이템을 반환합니다
def batch2TrainData(voc, pair_batch):
    pair_batch.sort(key=lambda x: len(x[0].split(" ")), reverse=True)
    input_batch, output_batch = [], []
    for pair in pair_batch:
        input_batch.append(pair[0])
        output_batch.append(pair[1])
    inp, lengths = inputVar(input_batch, voc)
    output, mask, max_target_len = outputVar(output_batch, voc)
    return inp, lengths, output, mask, max_target_len

In [8]:
class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = embedding

        # GRU를 초기화합니다. input_size와 hidden_size 패러미터는 둘 다 'hidden_size'로
        # 둡니다. 이는 우리 입력의 크기가 hideen_size 만큼의 피처를 갖는 단어 임베딩이기
        # 때문입니다.
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers,
                          dropout=(0 if n_layers == 1 else dropout), bidirectional=True)

    def forward(self, input_seq, input_lengths, hidden=None):
        # 단어 인덱스를 임베딩으로 변환합니다
        embedded = self.embedding(input_seq)
        # RNN 모듈을 위한 패딩된 배치 시퀀스를 패킹합니다
        packed = nn.utils.rnn.pack_padded_sequence(embedded, input_lengths, enforce_sorted=False)
        # GRU로 포워드 패스를 수행합니다
        outputs, hidden = self.gru(packed, hidden)
        # 패딩을 언패킹합니다
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs)
        # 양방향 GRU의 출력을 합산합니다
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:]
        # 출력과 마지막 은닉 상태를 반환합니다
        return outputs, hidden

In [9]:
# Luong 어텐션 레이어
class Attn(nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        if self.method not in ['dot', 'general', 'concat']:
            raise ValueError(self.method, "is not an appropriate attention method.")
        self.hidden_size = hidden_size
        if self.method == 'general':
            self.attn = nn.Linear(self.hidden_size, hidden_size)
        elif self.method == 'concat':
            self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
            self.v = nn.Parameter(torch.FloatTensor(hidden_size))

    def dot_score(self, hidden, encoder_output):
        return torch.sum(hidden * encoder_output, dim=2)

    def general_score(self, hidden, encoder_output):
        energy = self.attn(encoder_output)
        return torch.sum(hidden * energy, dim=2)

    def concat_score(self, hidden, encoder_output):
        energy = self.attn(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2)).tanh()
        return torch.sum(self.v * energy, dim=2)

    def forward(self, hidden, encoder_outputs):
        # Attention 가중치(에너지)를 제안된 방법에 따라 계산합니다
        if self.method == 'general':
            attn_energies = self.general_score(hidden, encoder_outputs)
        elif self.method == 'concat':
            attn_energies = self.concat_score(hidden, encoder_outputs)
        elif self.method == 'dot':
            attn_energies = self.dot_score(hidden, encoder_outputs)

        # max_length와 batch_size의 차원을 뒤집습니다
        attn_energies = attn_energies.t()

        # 정규화된 softmax 확률 점수를 반환합니다 (차원을 늘려서)
        return F.softmax(attn_energies, dim=1).unsqueeze(1)

In [10]:
class LuongAttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(LuongAttnDecoderRNN, self).__init__()

        # 참조를 보존해 둡니다
        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout

        # 레이어를 정의합니다
        self.embedding = embedding
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

        self.attn = Attn(attn_model, hidden_size)

    def forward(self, input_step, last_hidden, encoder_outputs):
        # 주의: 한 단위 시간에 대해 한 단계(단어)만을 수행합니다
        # 현재의 입력 단어에 대한 임베딩을 구합니다
        embedded = self.embedding(input_step)
        embedded = self.embedding_dropout(embedded)
        # 무방향 GRU로 포워드 패스를 수행합니다
        rnn_output, hidden = self.gru(embedded, last_hidden)
        # 현재의 GRU 출력을 바탕으로 어텐션 가중치를 계산합니다
        attn_weights = self.attn(rnn_output, encoder_outputs)
        # 인코더 출력에 어텐션을 곱하여 새로운 "가중치 합" 문백 벡터를 구합니다
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1))
        # Luong의 논문에 나온 식 5를 이용하여 가중치 문백 벡터와 GRU 출력을 결합합니다
        rnn_output = rnn_output.squeeze(0)
        context = context.squeeze(1)
        concat_input = torch.cat((rnn_output, context), 1)
        concat_output = torch.tanh(self.concat(concat_input))
        # Luong의 논문에 나온 식 6을 이용하여 다음 단어를 예측합니다
        output = self.out(concat_output)
        output = F.softmax(output, dim=1)
        # 출력과 마지막 은닉 상태를 반환합니다
        return output, hidden

In [11]:
def maskNLLLoss(inp, target, mask):
    nTotal = mask.sum()
    crossEntropy = -torch.log(torch.gather(inp, 1, target.view(-1, 1)).squeeze(1))
    loss = crossEntropy.masked_select(mask).mean()
    loss = loss.to(device)
    return loss, nTotal.item()

In [12]:
def train(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder, embedding,
          encoder_optimizer, decoder_optimizer, batch_size, clip, max_length=MAX_LENGTH):

    # 제로 그라디언트
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    # device 옵션을 설정합니다
    input_variable = input_variable.to(device)
    lengths = lengths.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)

    # 변수를 초기화합니다
    loss = 0
    print_losses = []
    n_totals = 0

    # 인코더로 포워드 패스를 수행합니다
    encoder_outputs, encoder_hidden = encoder(input_variable, lengths)

    # 초기 디코더 입력을 생성합니다(각 문장을 SOS 도큰으로 시작합니다)
    decoder_input = torch.LongTensor([[SOS_token for _ in range(batch_size)]])
    decoder_input = decoder_input.to(device)

    # 디코더의 초기 은닉 상태를 인코더의 마지막 은닉 상태로 둡니다
    decoder_hidden = encoder_hidden[:decoder.n_layers]

    # 이번 반복에서 teacher forcing을 사용할지를 결정합니다
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    # 배치 시퀀스를 한 번에 하나씩 디코더로 포워드 패스합니다
    if use_teacher_forcing:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # Teacher forcing 사용: 다음 입력을 현재의 목표로 둡니다
            decoder_input = target_variable[t].view(1, -1)
            # 손실을 계산하고 누적합니다
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal
    else:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # Teacher forcing 미사용: 다음 입력을 디코더의 출력으로 둡니다
            _, topi = decoder_output.topk(1)
            decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
            decoder_input = decoder_input.to(device)
            # 손실을 계산하고 누적합니다
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal

    # 역전파를 수행합니다
    loss.backward()

    # 그라디언트 클리핑: 그라디언트를 제자리에서 수정합니다
    _ = nn.utils.clip_grad_norm_(encoder.parameters(), clip)
    _ = nn.utils.clip_grad_norm_(decoder.parameters(), clip)

    # 모델의 가중치를 수정합니다
    encoder_optimizer.step()
    decoder_optimizer.step()

    return sum(print_losses) / n_totals

In [13]:
def trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer, embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size, print_every, save_every, clip, corpus_name, loadFilename):

    # 각 단계에 대한 배치를 읽어옵니다
    training_batches = [batch2TrainData(voc, [random.choice(pairs) for _ in range(batch_size)])
                      for _ in range(n_iteration)]

    # 초기화
    print('Initializing ...')
    start_iteration = 1
    print_loss = 0
    if loadFilename:
        start_iteration = checkpoint['iteration'] + 1

    # 학습 루프
    print("Training...")
    for iteration in range(start_iteration, n_iteration + 1):
        training_batch = training_batches[iteration - 1]
        # 배치에서 각 필드를 읽어옵니다
        input_variable, lengths, target_variable, mask, max_target_len = training_batch

        # 배치에 대해 학습을 한 단계 진행합니다
        loss = train(input_variable, lengths, target_variable, mask, max_target_len, encoder,
                     decoder, embedding, encoder_optimizer, decoder_optimizer, batch_size, clip)
        print_loss += loss

        # 경과를 출력합니다
        if iteration % print_every == 0:
            print_loss_avg = print_loss / print_every
            print("Iteration: {}; Percent complete: {:.1f}%; Average loss: {:.4f}".format(iteration, iteration / n_iteration * 100, print_loss_avg))
            print_loss = 0

        # Checkpoint를 저장합니다
        if (iteration % save_every == 0):
            directory = os.path.join(save_dir, model_name, corpus_name, '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size))
            if not os.path.exists(directory):
                os.makedirs(directory)
            torch.save({
                'iteration': iteration,
                'en': encoder.state_dict(),
                'de': decoder.state_dict(),
                'en_opt': encoder_optimizer.state_dict(),
                'de_opt': decoder_optimizer.state_dict(),
                'loss': loss,
                'voc_dict': voc.__dict__,
                'embedding': embedding.state_dict()
            }, os.path.join(directory, '{}_{}.tar'.format(iteration, 'checkpoint')))

In [14]:
class GreedySearchDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super(GreedySearchDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input_seq, input_length, max_length):
        # 인코더 모델로 입력을 포워드 패스합니다
        encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)
        # 인코더의 마지막 은닉 레이어가 디코더의 첫 번째 은닉 레이어의 입력이 되도록 준비합니다
        decoder_hidden = encoder_hidden[:decoder.n_layers]
        # 디코더의 첫 번째 입력을 SOS_token으로 초기화합니다
        decoder_input = torch.ones(1, 1, device=device, dtype=torch.long) * SOS_token
        # 디코더가 단어를 덧붙여 나갈 텐서를 초기화합니다
        all_tokens = torch.zeros([0], device=device, dtype=torch.long)
        all_scores = torch.zeros([0], device=device)
        # 반복적으로 각 단계마다 하나의 단어 토큰을 디코딩합니다
        for _ in range(max_length):
            # 디코더로의 포워드 패스를 수행합니다
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
            # 가장 가능성 높은 단어 토큰과 그 softmax 점수를 구합니다
            decoder_scores, decoder_input = torch.max(decoder_output, dim=1)
            # 토큰과 점수를 기록합니다
            all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
            all_scores = torch.cat((all_scores, decoder_scores), dim=0)
            # 현재의 토큰을 디코더의 다음 입력으로 준비시킵니다(차원을 증가시켜서)
            decoder_input = torch.unsqueeze(decoder_input, 0)
        # 단어 토큰과 점수를 모아서 반환합니다
        return all_tokens, all_scores

In [19]:
def evaluate(encoder, decoder, searcher, voc, sentence, max_length=MAX_LENGTH):
    ### 입력 시퀀스를 배치 형태로 만듭니다
    # 단어 -> 인덱스
    indexes_batch = [indexesFromSentence(voc, sentence)]
    # lengths 텐서를 만듭니다
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    # 배치의 차원을 뒤집어서 모델이 사용하는 형태로 만듭니다
    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
    # 적절한 디바이스를 사용합니다
    input_batch = input_batch.to(device)
    lengths = lengths.to(device)
    # searcher를 이용하여 문장을 디코딩합니다
    tokens, scores = searcher(input_batch, lengths, max_length)
    # 인덱스 -> 단어
    decoded_words = [voc.index2word[token.item()] for token in tokens]
    return decoded_words


def evaluateInput(encoder, decoder, searcher, voc):
    input_sentence = ''
    while(1):
        try:
            # 입력 문장을 받아옵니다
            input_sentence = input('> ')
            # 종료 조건인지 검사합니다
            if input_sentence == 'q' or input_sentence == 'quit': break
            # 문장을 정규화합니다
            input_sentence = normalizeString(input_sentence)
            # 문장을 평가합니다
            output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
            # 응답 문장을 형식에 맞춰 출력합니다
            output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
            print('Bot:', ' '.join(output_words))

        except KeyError:
            print("Error: 아직 학습중이라 똑똑하지 않아요. 다른 문장을 입력해주세요!")

In [16]:
# 모델을 설정합니다
model_name = 'cb_model'
attn_model = 'dot'
#attn_model = 'general'
#attn_model = 'concat'
hidden_size = 500
encoder_n_layers = 2
decoder_n_layers = 2
dropout = 0.1
batch_size = 64

# 불러올 checkpoint를 설정합니다. 처음부터 시작할 때는 None으로 둡니다.
loadFilename = None
checkpoint_iter = 5000
#loadFilename = os.path.join(save_dir, model_name, corpus_name,
#                            '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size),
#                            '{}_checkpoint.tar'.format(checkpoint_iter))


# loadFilename이 제공되는 경우에는 모델을 불러옵니다
if loadFilename:
    # 모델을 학습할 때와 같은 기기에서 불러오는 경우
    checkpoint = torch.load(loadFilename)
    # GPU에서 학습한 모델을 CPU로 불러오는 경우
    #checkpoint = torch.load(loadFilename, map_location=torch.device('cpu'))
    encoder_sd = checkpoint['en']
    decoder_sd = checkpoint['de']
    encoder_optimizer_sd = checkpoint['en_opt']
    decoder_optimizer_sd = checkpoint['de_opt']
    embedding_sd = checkpoint['embedding']
    voc.__dict__ = checkpoint['voc_dict']


print('Building encoder and decoder ...')
# 단어 임베딩을 초기화합니다
embedding = nn.Embedding(voc.num_words, hidden_size)
if loadFilename:
    embedding.load_state_dict(embedding_sd)
# 인코더 및 디코더 모델을 초기화합니다
encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)
if loadFilename:
    encoder.load_state_dict(encoder_sd)
    decoder.load_state_dict(decoder_sd)
# 적절한 디바이스를 사용합니다
encoder = encoder.to(device)
decoder = decoder.to(device)
print('Models built and ready to go!')

Building encoder and decoder ...
Models built and ready to go!


In [17]:
# 학습 및 최적화 설정
clip = 50.0
teacher_forcing_ratio = 1.0
learning_rate = 0.0001
decoder_learning_ratio = 5.0
n_iteration = 5000
print_every = 1
save_every = 500

# Dropout 레이어를 학습 모드로 둡니다
encoder.train()
decoder.train()

# Optimizer를 초기화합니다
print('Building optimizers ...')
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
if loadFilename:
    encoder_optimizer.load_state_dict(encoder_optimizer_sd)
    decoder_optimizer.load_state_dict(decoder_optimizer_sd)

# cuda가 있다면 cuda를 설정합니다
for state in encoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()

for state in decoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()
    
# 학습 단계를 수행합니다
print("Starting Training!")
trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer,
           embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size,
           print_every, save_every, clip, corpus_name, loadFilename)

Building optimizers ...
Starting Training!
Initializing ...
Training...


<ipython-input-11-f319a975feb5>:4: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  ..\aten\src\ATen/native/IndexingUtils.h:20.)
  loss = crossEntropy.masked_select(mask).mean()


Iteration: 1; Percent complete: 0.0%; Average loss: 8.6494
Iteration: 2; Percent complete: 0.0%; Average loss: 8.5729
Iteration: 3; Percent complete: 0.1%; Average loss: 8.4893
Iteration: 4; Percent complete: 0.1%; Average loss: 8.3350
Iteration: 5; Percent complete: 0.1%; Average loss: 8.1452
Iteration: 6; Percent complete: 0.1%; Average loss: 7.8019
Iteration: 7; Percent complete: 0.1%; Average loss: 7.4726
Iteration: 8; Percent complete: 0.2%; Average loss: 7.1238
Iteration: 9; Percent complete: 0.2%; Average loss: 6.9820
Iteration: 10; Percent complete: 0.2%; Average loss: 6.8259
Iteration: 11; Percent complete: 0.2%; Average loss: 6.5439
Iteration: 12; Percent complete: 0.2%; Average loss: 6.1661
Iteration: 13; Percent complete: 0.3%; Average loss: 6.1196
Iteration: 14; Percent complete: 0.3%; Average loss: 5.9169
Iteration: 15; Percent complete: 0.3%; Average loss: 5.8305
Iteration: 16; Percent complete: 0.3%; Average loss: 5.6615
Iteration: 17; Percent complete: 0.3%; Average lo

Iteration: 138; Percent complete: 2.8%; Average loss: 3.5562
Iteration: 139; Percent complete: 2.8%; Average loss: 3.5263
Iteration: 140; Percent complete: 2.8%; Average loss: 3.5573
Iteration: 141; Percent complete: 2.8%; Average loss: 3.4709
Iteration: 142; Percent complete: 2.8%; Average loss: 3.4533
Iteration: 143; Percent complete: 2.9%; Average loss: 3.7014
Iteration: 144; Percent complete: 2.9%; Average loss: 3.5731
Iteration: 145; Percent complete: 2.9%; Average loss: 3.0958
Iteration: 146; Percent complete: 2.9%; Average loss: 3.2388
Iteration: 147; Percent complete: 2.9%; Average loss: 3.6357
Iteration: 148; Percent complete: 3.0%; Average loss: 3.7309
Iteration: 149; Percent complete: 3.0%; Average loss: 3.3590
Iteration: 150; Percent complete: 3.0%; Average loss: 3.4240
Iteration: 151; Percent complete: 3.0%; Average loss: 3.3049
Iteration: 152; Percent complete: 3.0%; Average loss: 3.4871
Iteration: 153; Percent complete: 3.1%; Average loss: 3.5025
Iteration: 154; Percent 

Iteration: 273; Percent complete: 5.5%; Average loss: 2.9733
Iteration: 274; Percent complete: 5.5%; Average loss: 3.0371
Iteration: 275; Percent complete: 5.5%; Average loss: 2.8897
Iteration: 276; Percent complete: 5.5%; Average loss: 3.0027
Iteration: 277; Percent complete: 5.5%; Average loss: 2.9903
Iteration: 278; Percent complete: 5.6%; Average loss: 3.0402
Iteration: 279; Percent complete: 5.6%; Average loss: 2.9427
Iteration: 280; Percent complete: 5.6%; Average loss: 2.6284
Iteration: 281; Percent complete: 5.6%; Average loss: 2.9000
Iteration: 282; Percent complete: 5.6%; Average loss: 2.8320
Iteration: 283; Percent complete: 5.7%; Average loss: 3.1164
Iteration: 284; Percent complete: 5.7%; Average loss: 2.9916
Iteration: 285; Percent complete: 5.7%; Average loss: 3.2356
Iteration: 286; Percent complete: 5.7%; Average loss: 2.8596
Iteration: 287; Percent complete: 5.7%; Average loss: 3.2056
Iteration: 288; Percent complete: 5.8%; Average loss: 2.8028
Iteration: 289; Percent 

...

Iteration: 4347; Percent complete: 86.9%; Average loss: 0.5630
Iteration: 4348; Percent complete: 87.0%; Average loss: 0.5510
Iteration: 4349; Percent complete: 87.0%; Average loss: 0.5290
Iteration: 4350; Percent complete: 87.0%; Average loss: 0.5916
Iteration: 4351; Percent complete: 87.0%; Average loss: 0.5867
Iteration: 4352; Percent complete: 87.0%; Average loss: 0.6128
Iteration: 4353; Percent complete: 87.1%; Average loss: 0.4877
Iteration: 4354; Percent complete: 87.1%; Average loss: 0.5573
Iteration: 4355; Percent complete: 87.1%; Average loss: 0.5373
Iteration: 4356; Percent complete: 87.1%; Average loss: 0.5525
Iteration: 4357; Percent complete: 87.1%; Average loss: 0.4970
Iteration: 4358; Percent complete: 87.2%; Average loss: 0.5390
Iteration: 4359; Percent complete: 87.2%; Average loss: 0.5375
Iteration: 4360; Percent complete: 87.2%; Average loss: 0.5774
Iteration: 4361; Percent complete: 87.2%; Average loss: 0.5263
Iteration: 4362; Percent complete: 87.2%; Average loss:

Iteration: 4478; Percent complete: 89.6%; Average loss: 0.5234
Iteration: 4479; Percent complete: 89.6%; Average loss: 0.5657
Iteration: 4480; Percent complete: 89.6%; Average loss: 0.5994
Iteration: 4481; Percent complete: 89.6%; Average loss: 0.5048
Iteration: 4482; Percent complete: 89.6%; Average loss: 0.4466
Iteration: 4483; Percent complete: 89.7%; Average loss: 0.5110
Iteration: 4484; Percent complete: 89.7%; Average loss: 0.4921
Iteration: 4485; Percent complete: 89.7%; Average loss: 0.5917
Iteration: 4486; Percent complete: 89.7%; Average loss: 0.5514
Iteration: 4487; Percent complete: 89.7%; Average loss: 0.5499
Iteration: 4488; Percent complete: 89.8%; Average loss: 0.5452
Iteration: 4489; Percent complete: 89.8%; Average loss: 0.5486
Iteration: 4490; Percent complete: 89.8%; Average loss: 0.5183
Iteration: 4491; Percent complete: 89.8%; Average loss: 0.4899
Iteration: 4492; Percent complete: 89.8%; Average loss: 0.5505
Iteration: 4493; Percent complete: 89.9%; Average loss:

Iteration: 4609; Percent complete: 92.2%; Average loss: 0.5017
Iteration: 4610; Percent complete: 92.2%; Average loss: 0.5273
Iteration: 4611; Percent complete: 92.2%; Average loss: 0.4683
Iteration: 4612; Percent complete: 92.2%; Average loss: 0.3914
Iteration: 4613; Percent complete: 92.3%; Average loss: 0.4732
Iteration: 4614; Percent complete: 92.3%; Average loss: 0.4626
Iteration: 4615; Percent complete: 92.3%; Average loss: 0.4685
Iteration: 4616; Percent complete: 92.3%; Average loss: 0.4547
Iteration: 4617; Percent complete: 92.3%; Average loss: 0.4819
Iteration: 4618; Percent complete: 92.4%; Average loss: 0.4759
Iteration: 4619; Percent complete: 92.4%; Average loss: 0.4619
Iteration: 4620; Percent complete: 92.4%; Average loss: 0.4904
Iteration: 4621; Percent complete: 92.4%; Average loss: 0.4655
Iteration: 4622; Percent complete: 92.4%; Average loss: 0.4757
Iteration: 4623; Percent complete: 92.5%; Average loss: 0.4355
Iteration: 4624; Percent complete: 92.5%; Average loss:

Iteration: 4740; Percent complete: 94.8%; Average loss: 0.4572
Iteration: 4741; Percent complete: 94.8%; Average loss: 0.5101
Iteration: 4742; Percent complete: 94.8%; Average loss: 0.5520
Iteration: 4743; Percent complete: 94.9%; Average loss: 0.5089
Iteration: 4744; Percent complete: 94.9%; Average loss: 0.4286
Iteration: 4745; Percent complete: 94.9%; Average loss: 0.5335
Iteration: 4746; Percent complete: 94.9%; Average loss: 0.3837
Iteration: 4747; Percent complete: 94.9%; Average loss: 0.3970
Iteration: 4748; Percent complete: 95.0%; Average loss: 0.4340
Iteration: 4749; Percent complete: 95.0%; Average loss: 0.4008
Iteration: 4750; Percent complete: 95.0%; Average loss: 0.3494
Iteration: 4751; Percent complete: 95.0%; Average loss: 0.5205
Iteration: 4752; Percent complete: 95.0%; Average loss: 0.4945
Iteration: 4753; Percent complete: 95.1%; Average loss: 0.4588
Iteration: 4754; Percent complete: 95.1%; Average loss: 0.5925
Iteration: 4755; Percent complete: 95.1%; Average loss:

Iteration: 4871; Percent complete: 97.4%; Average loss: 0.4387
Iteration: 4872; Percent complete: 97.4%; Average loss: 0.4787
Iteration: 4873; Percent complete: 97.5%; Average loss: 0.4093
Iteration: 4874; Percent complete: 97.5%; Average loss: 0.3863
Iteration: 4875; Percent complete: 97.5%; Average loss: 0.3985
Iteration: 4876; Percent complete: 97.5%; Average loss: 0.4010
Iteration: 4877; Percent complete: 97.5%; Average loss: 0.4145
Iteration: 4878; Percent complete: 97.6%; Average loss: 0.3785
Iteration: 4879; Percent complete: 97.6%; Average loss: 0.4122
Iteration: 4880; Percent complete: 97.6%; Average loss: 0.4324
Iteration: 4881; Percent complete: 97.6%; Average loss: 0.4232
Iteration: 4882; Percent complete: 97.6%; Average loss: 0.5876
Iteration: 4883; Percent complete: 97.7%; Average loss: 0.3459
Iteration: 4884; Percent complete: 97.7%; Average loss: 0.4526
Iteration: 4885; Percent complete: 97.7%; Average loss: 0.4467
Iteration: 4886; Percent complete: 97.7%; Average loss:

In [20]:
# Dropout 레이어를 평가 모드로 설정합니다
encoder.eval()
decoder.eval()

# 탐색 모듈을 초기화합니다
searcher = GreedySearchDecoder(encoder, decoder)

# 채팅을 시작합니다 (다음 줄의 주석을 제거하면 시작해볼 수 있습니다)
evaluateInput(encoder, decoder, searcher, voc)

> 안녕하세요?
Bot: 살짝 는 못 쓰 게 되 ㄴ다면 솔직 하 게 고백 하 아 보 시 어요 . 죠 ?
> 안녕하세요.
Bot: 안녕하세요 . 맘 을 먹 는 건 어떻 ㄹ까요 ? 하 어요 . 지 않 을까요 ?
> 안녕하세요.
Bot: 안녕하세요 . 맘 을 먹 는 건 어떻 ㄹ까요 ? 하 어요 . 지 않 을까요 ?
> 만나서 반가워요.
Bot: 너무 좋아하 았 던 게 아니 라면 잘못 의 사랑 을 좋아하 나 보 아요 . 었 나요 .
> 살고싶지않아요.
Bot: 당신 은 당신 의 마음 을 읽 을 수 있 는 거 예요 . 지 않 을까요 ?
> 보고싶어요.
Bot: 보 고 싶 은 건 없 나요 . 거 예요 . 지 ㄴ 않 을 거 예요 .
> 보 고 싶 어 요.
Bot: 친구 들 과 연락 하 아 보 시 어요 . 었 나요 ? . 되 ㅂ니다
> quit
